In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
from collections import deque
import time # サーバー負荷軽減のため

In [6]:
# 定数設定
BASE_URL = "https://www.musashino-u.ac.jp/"
DOMAIN_CHECK = "musashino-u.ac.jp" 

# User-Agentの設定
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 巡回とデータ格納用の変数
crawled_data = {}        
url_queue = deque([BASE_URL]) 
visited_urls = set()

In [ ]:
# 巡回とデータ収集
while url_queue:
    current_url = url_queue.popleft()

    if current_url in visited_urls:
        continue
    
    print(f"--- 巡回中: {current_url}")
    
    visited_urls.add(current_url) 
    time.sleep(1) # サーバー負荷軽減
    
    try:
        # headers=HEADERS
        response = requests.get(current_url, headers=HEADERS) 
        response.encoding = response.apparent_encoding

        soup = BeautifulSoup(response.text, 'html.parser')

        #　<title>とURLを辞書に格納
        title_tag = soup.find('title')
        title_content = str(title_tag) if title_tag else "<title>タイトルなし</title>"
        
        crawled_data[current_url] = title_content

        # 同一ドメイン内のリンクを抽出
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            
            if href.startswith('/'):
                base_parts = current_url.split('/')
                base_url_path = f"{base_parts[0]}//{base_parts[2]}"
                absolute_url = base_url_path + href
            elif href.startswith('http'):
                absolute_url = href
            else:
                continue

            if DOMAIN_CHECK in absolute_url:
                clean_url = absolute_url.split('#')[0].split('?')[0]
                
                if clean_url not in visited_urls and clean_url not in url_queue:
                    url_queue.append(clean_url)
            
    except Exception as e:
        print(f"アクセスエラー: {current_url} ({e})")
        continue

    # 辞書に格納されたデータ数が30件を超えたらループを終了する
    if len(crawled_data) >= 30:
        print("\n収集制限に達したため、巡回を停止します")
        break

--- 巡回中: https://www.musashino-u.ac.jp/
--- 巡回中: https://ef.musashino-u.ac.jp/donation/
--- 巡回中: https://www.musashino-u.ac.jp/access.html
--- 巡回中: https://www.musashino-u.ac.jp/admission/request.html
--- 巡回中: https://www.musashino-u.ac.jp/contact.html
--- 巡回中: https://www.musashino-u.ac.jp/prospective-students.html
--- 巡回中: https://www.musashino-u.ac.jp/students.html
--- 巡回中: https://www.musashino-u.ac.jp/alumni.html
--- 巡回中: https://www.musashino-u.ac.jp/parents.html
--- 巡回中: https://www.musashino-u.ac.jp/business.html
--- 巡回中: https://www.musashino-u.ac.jp/guide/
--- 巡回中: https://www.musashino-u.ac.jp/guide/profile/
--- 巡回中: https://www.musashino-u.ac.jp/guide/activities/
--- 巡回中: https://www.musashino-u.ac.jp/guide/campus/
--- 巡回中: https://www.musashino-u.ac.jp/guide/facility/
--- 巡回中: https://www.musashino-u.ac.jp/guide/information/
--- 巡回中: https://www.musashino-u.ac.jp/guide/profile/media/
--- 巡回中: https://www.musashino-u.ac.jp/admission/
--- 巡回中: https://www.musashino-u.ac.jp/a

In [ ]:
# 最終結果の表示
print(f"巡回終了: 収集データ数 {len(crawled_data)} 件")

# 辞書型変数を print() で表示する
print(crawled_data)

✅ 巡回終了: 収集データ数 30 件
{'https://www.musashino-u.ac.jp/': '<title>武蔵野大学</title>', 'https://ef.musashino-u.ac.jp/donation/': '<title>ご寄付のお願い | 学校法人武蔵野大学</title>', 'https://www.musashino-u.ac.jp/access.html': '<title>交通アクセス | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/admission/request.html': '<title>資料請求 | 入試情報 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/contact.html': '<title>お問い合わせ | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/prospective-students.html': '<title>武蔵野大学で学びたい方 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/students.html': '<title>在学生の方 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/alumni.html': '<title>卒業生の方 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/parents.html': '<title>保護者の方 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/business.html': '<title>企業・研究者の方 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/guide/': '<title>大学案内 | 武蔵野大学</title>', 'https://www.musashino-u.ac.jp/guide/profile/': '<title>大学紹介 | 大学案内 | 武蔵野大学</title>', 'https://www.musashi